In [40]:
import pandas as pd
import numpy as np
import sklearn

inputs = 'C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/files/NonCodify/12 - December 2022 Combined Inputs.xlsx'
outputs = 'C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/files/NonCodify/12 - December 2022 - Combined Outputs.xlsx'
pics = 'C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/files/12 - December 2022 All PICs.txt'
shs = 'C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/files/Sutherland.xlsx'

inputs = pd.read_excel(inputs, engine='openpyxl')
outputs = pd.read_excel(outputs, engine='openpyxl')
shs = pd.read_excel(shs, engine = 'openpyxl')
pics = pd.read_csv(pics, sep = '\t', header=None)
pics.drop(columns=[1,2,3,4], inplace=True)
pics.columns = ['Invoice', 'Post Date', 'Code']
pics['Post Date'] = pd.to_datetime(pics['Post Date'])
pic_desc = pd.read_excel('C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/Dictionary 6 - RPA Only '
                         'Rejections.xlsx', engine='openpyxl')
pics = pics.merge(pic_desc, how='left')
pics = pics[pics['Bot Name'].str.contains('Bundling')]
df = pd.merge(inputs, shs, how='left',
              left_on = ['INVNUM', 'File Date', 'SHS Name'],
              right_on=['INVNUM', 'BOTRequestDate', 'BotName'])
df = pd.merge(df, pics, how='left',
              left_on = ['INVNUM', 'LastModifiedDate'],
              right_on = ['Invoice', 'Post Date'])

In [41]:
outputs['Process'] = outputs['Process'].str.strip()
df = pd.merge(df, outputs, how='left',
              left_on = ['INVNUM', 'File Date', 'Coding Tool'],
              right_on = ['Invoice', 'File Date', 'Process'])
df.drop(columns=['SHS Name', 'Invoice_x', 'Bot Type', 'Bot Name', 'Invoice_y', 'Process'], inplace=True)

In [43]:
inputs = ''
outputs = ''
shs = ''
pic_desc = ''
pics = ''
data = df.copy()
df = ''

In [45]:
from sklearn.preprocessing import LabelEncoder

le_payer = LabelEncoder()
le_coding_tool = LabelEncoder()
le_shs_status = LabelEncoder()
le_pic_desc = LabelEncoder()
le_category = LabelEncoder()
le_status = LabelEncoder()
le_comment = LabelEncoder()

In [63]:
df = data[['ACK', 'Form Letter', 'Proof']]
df = df.rename({'Proof': 'Proofs'}, axis='columns')
df = df.fillna(0)

In [64]:
df['ACK'] = df['ACK'].astype('int64')
df['Form Letter'] = df['Form Letter'].astype('int64')
df['Proofs'] = df['Proofs'].astype('int64')

df['payer_n'] = le_payer.fit_transform(data['PAYER'])
df['coding_tool_n'] = le_coding_tool.fit_transform(data['Coding Tool'])
df['shs_status_n'] = le_shs_status.fit_transform(data['RetrievalStatus'])
df['pic_desc_n'] = le_shs_status.fit_transform(data['Name'])

In [72]:
import joblib

model = joblib.load('C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/ML Training/models/noncodify_audit_status.joblib')

In [73]:
predictions = le_status.inverse_transform(model.predict(df))

NotFittedError: This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.